# Installation

* Install Anaconda
* Create environment with Python 3.7.x
* Start terminal with Environment and run:
    * conda install -c esri arcgis=1.8.2
    * conda install nodejs=10.13.0
    * conda install jupyterlab=2.1.5
* Run: juypter lab
* Click on Extension button and enable 3rd party app installation
* Install the following extensions and rebuild:
    * Jupyterlab-manager
    * Arcgis-map-ipywidget
* References: 
    * https://developers.arcgis.com/python/guide/
    * https://developers.arcgis.com/python/guide/visualizing-data-with-the-spatially-enabled-dataframe/
    * https://developers.arcgis.com/python/guide/using-the-jupyter-lab-environment/
    * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html
    * https://developers.arcgis.com/documentation/common-data-types/renderer-objects.htm
    * https://developers.arcgis.com/arcade/

In [1]:
# Obtain the GeoJSON data
# https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

import requests

# url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"
data = requests.get(url)

json = data.json()

# Show summary of what was read
json["metadata"]



{'generated': 1654191120000,
 'url': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson',
 'title': 'USGS All Earthquakes, Past Month',
 'status': 200,
 'api': '1.10.3',
 'count': 8747}

In [2]:
# Convert the GeoJSON data to a FeatureSet (ArcGIS) and then to a Spatial Data Frame (Pandas)

from arcgis.features import FeatureSet
import numpy as np
import pandas as pd

# For simplicity, ignore this warning
pd.set_option('mode.chained_assignment', None)

# Convert json to ArcGIS feature Set
fs = FeatureSet.from_geojson(json)

# Convert ArcGIS feature set to spatial data frame compatible with Pandas
sdf = fs.sdf

# Cleanup data frame
sdf = sdf.replace(np.nan,0)  # Cleanup Invalid Numbers
sdf.time = sdf.time.apply(lambda x : x // 1000) # Convert times to seconds for Pandas
sdf.updated = sdf.updated.apply(lambda x : x // 1000)
sdf = sdf.query("mag > 0")

sdf


,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,types,nst,dmin,rms,gap,magType,type,title,OBJECTID,SHAPE
0,1.97,"2 km SSW of Oologah, Oklahoma",1654190679,1654190867,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",origin,phase-data,",16.0,0.566359,1.118974,253.998535,ml,earthquake,"M 2.0 - 2 km SSW of Oologah, Oklahoma",1,"{""x"": -95.71632385, ""y"": 36.42536163, ""spatial..."
1,2.00,"12 km N of Yauco, Puerto Rico",1654186346,1654187229,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",origin,phase-data,",5.0,0.000000,0.190000,307.000000,md,earthquake,"M 2.0 - 12 km N of Yauco, Puerto Rico",2,"{""x"": -66.8508333333333, ""y"": 18.1483333333333..."
2,1.40,"34 km S of Denali National Park, Alaska",1654185672,1654185812,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",origin,phase-data,",0.0,0.000000,0.750000,0.000000,ml,earthquake,"M 1.4 - 34 km S of Denali National Park, Alaska",3,"{""x"": -151.7338, ""y"": 63.2308, ""spatialReferen..."
3,0.74,"4km SE of Beaumont, CA",1654185482,1654185702,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",nearby-cities,origin,phase-data,scitech-link,",12.0,0.206800,0.180000,107.000000,ml,earthquake,"M 0.7 - 4km SE of Beaumont, CA",4,"{""x"": -116.9441667, ""y"": 33.9098333, ""spatialR..."
4,1.30,"25 km S of Susitna, Alaska",1654184949,1654185410,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",origin,phase-data,",0.0,0.000000,0.820000,0.000000,ml,earthquake,"M 1.3 - 25 km S of Susitna, Alaska",5,"{""x"": -150.5517, ""y"": 61.316, ""spatialReferenc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,0.21,"38 km NNE of Amboy, Washington",1651600949,1651796687,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",origin,phase-data,",14.0,0.011770,0.220000,148.000000,ml,earthquake,"M 0.2 - 38 km NNE of Amboy, Washington",8743,"{""x"": -122.17433333333334, ""y"": 46.19716666666..."
8743,1.69,"45km NE of Holtville, CA",1651600906,1651603733,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",nearby-cities,origin,phase-data,scitech-link,",14.0,0.082220,0.270000,80.000000,ml,quarry blast,"M 1.7 Quarry Blast - 45km NE of Holtville, CA",8744,"{""x"": -114.9966667, ""y"": 33.0531667, ""spatialR..."
8744,0.84,"6km NW of The Geysers, CA",1651600557,1651603037,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",nearby-cities,origin,phase-data,scitech-link,",16.0,0.006175,0.020000,64.000000,md,earthquake,"M 0.8 - 6km NW of The Geysers, CA",8745,"{""x"": -122.8021698, ""y"": 38.8235016, ""spatialR..."
8745,1.24,"6 km ENE of Clayton, Idaho",1651599335,1651601651,0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0.0,0.0,0.0,...,",origin,phase-data,",7.0,0.232000,0.090000,145.000000,ml,earthquake,"M 1.2 - 6 km ENE of Clayton, Idaho",8746,"{""x"": -114.322, ""y"": 44.271, ""spatialReference..."


In [3]:
from arcgis import GIS

# Simple map with earthquakes plotted

m1 = GIS().map("United States", zoomlevel=3)
m1.center = [39, -98]
m1.basemap = 'topo-vector'
sdf.spatial.plot(map_widget=m1)
m1

MapView(layout=Layout(height='400px', width='100%'))

In [32]:
# Modify the markers

m2 = GIS().map("United States", zoomlevel=3)
m2.center = [39, -98]
m2.basemap = 'gray'
sdf.spatial.plot(map_widget=m2,
                 renderer_type='s', 
                 symbol_type='simple',
                 symbol_style='d',
                 colors= [247,101,76, 255],
                 line_width= 0,
                 marker_size=10)

m2

MapView(layout=Layout(height='400px', width='100%'))

In [34]:
# Overlay another feature layer

from arcgis.features import FeatureLayer

m3 = GIS().map("United States", zoomlevel=3)
m3.center = [39, -98]
m3.basemap = 'gray'



sdf.spatial.plot(map_widget=m3,
                 renderer_type='s', 
                 symbol_type='simple',
                 symbol_style='o',
                 colors= [247,101,76, 255],
                 line_width= 0,
                 marker_size=5)

fault_url = "https://services.arcgis.com/nzS0F0zdNLvs7nc8/arcgis/rest/services/Sean_View_6/FeatureServer/0"
fault_layer = FeatureLayer(fault_url)
m3.add_layer(fault_layer)

m3.legend = True
m3

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [31]:
# Different colors (autogenerated) for different magnitudes

m4 = GIS().map("United States", zoomlevel=3)
m4.center = [39, -98]
m4.basemap = 'gray'
sdf.spatial.plot(map_widget=m4,
               renderer_type='c',  # for class breaks renderer
               method='esriClassifyNaturalBreaks',  # classification algorithm
               class_count=5,  # choose the number of classes
               col='mag',  # numeric column to classify
               line_width=1
               )

m4.legend = True
m4

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [30]:
# Different colors (manual) for different magnitudes

from arcgis.mapping import renderer

m5= GIS().map("United States", zoomlevel=3)
m5.center = [39, -98]
m5.basemap = 'gray'

m5_renderer = renderer.generate_renderer("Point",sdf_or_series=sdf,render_type="c", field="mag")

# {'classMaxValue': 2,
#        'label': '0-2',
#        'symbol': {'type': 'esriSMS',
#             'size' : 1,                                       
#             'style': 'esriSMSCircle',
#             'color': [67,130,72, 255]}},
#
# 0-2 : [67,130,72, 255]
# 2-4 : [126,202,206, 255]
# 4-6 : [235,247,76, 255]
# > 6 : [247,101,76, 255]

m5_renderer["classBreakInfos"] = [{'classMaxValue': 2,
       'label': '0-2',
       'symbol': {'type': 'esriSMS',
            'size' : 1,                                       
            'style': 'esriSMSCircle',
            'color': [67,130,72, 255]}},
    {'classMaxValue': 4,
       'label': '2-4',
       'symbol': {'type': 'esriSMS',
            'size' : 3,                  
            'style': 'esriSMSCircle',
            'color': [126,202,206, 255]}},
    {'classMaxValue': 6,
       'label': '4-6',
       'symbol': {'type': 'esriSMS',
           'size' : 5,                  
           'style': 'esriSMSCircle',
           'color': [235,247,76, 255]}},
    {'classMaxValue': 10,
       'label': '> 6',
       'symbol': {'type': 'esriSMS',
            'size' : 10,                  
           'style': 'esriSMSCircle',
           'color': [247,101,76, 255]}}]
  

sdf.spatial.plot(map_widget=m5, renderer=m5_renderer)

m5.legend = True
m5

MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [33]:
# Heatmap

m6 = GIS().map("United States", zoomlevel=3)
m6.center = [39, -98]
m6.basemap = 'gray'

sdf_larger = sdf.query("mag >= 3")

m6_renderer = renderer.generate_renderer("Point",sdf_or_series=sdf_larger,render_type="h", blur_radius=5, stops=10)

sdf_larger.spatial.plot(map_widget=m6,
                 renderer=m6_renderer)

m6.legend = True
m6


MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [39]:
# Unique Values (Time) and Arcade Expressions

from datetime import datetime

m7 = GIS().map("United States", zoomlevel=3)
m7.center = [39, -98]
m7.basemap = 'gray'

now = datetime.today().timestamp()
sdf["age_days"] = (now - sdf.time) // 86400 # Seconds in a day

# sdf["recent"] = sdf.apply(lambda r : "Today" if r.age_days < 1 else ("Week" if r.age_days < 7 else "Month"), axis=1)

recent = '''
if ($feature.age_days < 1) {
    return "Today"
} else if ($feature.age_days < 7) {
    return "Week"
} else {
    return "Month"
}
'''

recent_values = [
    { "value": "Today", 
      "label": "Today",
      "symbol": {'type': 'esriSMS',
            'style': 'esriSMSCircle',
            'size' : 10,
            'color': [247,101,76, 255]}},
    { "value": "Week", 
      "label": "Week",
      "symbol": {'type': 'esriSMS',
            'style': 'esriSMSCircle',
            'size': 5,
            'color': [126,202,206, 255]}},    
    { "value": "Month", 
      "label": "Month",
      "symbol": {'type': 'esriSMS',
            'style': 'esriSMSCircle',
            'size' : 2,  
            'color': [235,247,76, 255]}}]


sdf.spatial.plot(map_widget=m7,
               renderer_type='u-a',  # for unique value rendering
               unique_values=recent_values,
               arcade_expression=recent,  # category column
               )

m7.legend = True
m7


MapView(layout=Layout(height='400px', width='100%'), legend=True)